In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
# import ydf
import tensorflow_decision_forests as tfdf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs found: {gpus}")
else:
    print("No GPUs found.")

2025-10-06 16:09:09.376592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759766949.636817      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759766949.710984      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


No GPUs found.


2025-10-06 16:09:30.148031: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
train_df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
# preprocess?
def preprocess(df):
    # def cryoSleep_integer(x):
    #     return x.astype(int)
    # df=df.dropna()
    def normalize_name(x):
        x=str(x)
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")][1:])
    def split_passenger_id(row):
        if pd.isna(row['PassengerId']):
            return pd.Series([None, None, None])
        arr=row['PassengerId'].split('_')
        if len(arr) < 2:
            arr += [None] * (3 - len(arr))
        
        return pd.Series(arr)
    def split_cabin(row):
        if pd.isna(row['Cabin']):
            return pd.Series([None, None, None])  # Handle None case
        arr = row['Cabin'].split('/')
        # Ensure arr has exactly 3 elements
        if len(arr) < 3:
            arr += [None] * (3 - len(arr))  # Pad with None
        return pd.Series(arr)
    def total_spent(row):
        return pd.Series(row['RoomService']+row['FoodCourt']+row['ShoppingMall']+row['Spa']+row['VRDeck'])
        

    # Apply the function and create new columns
    df['Total_Spent']=df.apply(total_spent,axis=1)
    df["Name"] = df["Name"].apply(normalize_name)
    df[['Deck', 'Num', 'Side']] = df.apply(split_cabin, axis=1)
    df[['Group','Place_In_Group']]=df.apply(split_passenger_id,axis=1)
    df['CryoSleep'] = df['CryoSleep'].apply(lambda x: float(x))
    df['VIP'] = df['VIP'].apply(lambda x: float(x))
    
    return df
    
train_df=preprocess(train_df)
test_df=preprocess(test_df)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     8693 non-null   object 
 1   HomePlanet      8492 non-null   object 
 2   CryoSleep       8476 non-null   float64
 3   Cabin           8494 non-null   object 
 4   Destination     8511 non-null   object 
 5   Age             8514 non-null   float64
 6   VIP             8490 non-null   float64
 7   RoomService     8512 non-null   float64
 8   FoodCourt       8510 non-null   float64
 9   ShoppingMall    8485 non-null   float64
 10  Spa             8510 non-null   float64
 11  VRDeck          8505 non-null   float64
 12  Name            8693 non-null   object 
 13  Transported     8693 non-null   bool   
 14  Total_Spent     7785 non-null   float64
 15  Deck            8494 non-null   object 
 16  Num             8494 non-null   object 
 17  Side            8494 non-null   o

In [5]:
train_df.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Total_Spent,Deck,Num,Side,Group,Place_In_Group
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Ofracculy,False,0.0,B,0,P,0001,01
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Vines,True,736.0,F,0,S,0002,01
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Susent,False,10383.0,A,0,S,0003,01
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Susent,False,5176.0,A,0,S,0003,02
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Santantines,True,1091.0,F,1,S,0004,01


In [6]:
input_features=list(train_df.columns)
input_features.remove('Transported')
input_features.remove('PassengerId')
input_features.remove('Cabin')
# input_features.remove('Name')
# imput_features.remove('')

In [7]:
def tokenize_names(features, labels=None):
    features['Name']=tf.strings.split(features['Name'])
    return features, labels

train_ds=tfdf.keras.pd_dataframe_to_tf_dataset(train_df,label="Transported").map(tokenize_names)
test_ds=tfdf.keras.pd_dataframe_to_tf_dataset(test_df).map(tokenize_names)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

I0000 00:00:1759766981.484359      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759766981.485731      13 kernel.cc:783] Collect training examples
I0000 00:00:1759766981.485764      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^HomePlanet$"
}
column_guides {
  column_name_pattern: "^CryoSleep$"
}
column_guides {
  column_name_pattern: "^Destination$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^VIP$"
}
column_guides {
  column_name_pattern: "^RoomService$"
}
column_guides {
  column_name_pattern: "^FoodCourt$"
}
column_guides {
  column_name_pattern: "^ShoppingMall$"
}
column_guides {
  column_name_pattern: "^Spa$"
}
column_guides {
  column_name_pattern: "^VRDeck$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern:

Accuracy: 0.7770419716835022 Loss:0.89658522605896


In [9]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (17):
	Age
	CryoSleep
	Deck
	Destination
	FoodCourt
	Group
	HomePlanet
	Name
	Num
	Place_In_Group
	RoomService
	ShoppingMall
	Side
	Spa
	Total_Spent
	VIP
	VRDeck

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Name"  0.308676 ################
    2.      "CryoSleep"  0.229726 ######
    3.            "Spa"  0.218254 #####
    4.            "Num"  0.215261 #####
    5.         "VRDeck"  0.206933 ####
    6.           "Deck"  0.199930 ###
    7.          "Group"  0.198071 ###
    8.    "RoomService"  0.196642 ###
    9.            "Age"  0.184864 #
   10.

In [10]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(test_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Transported": (proba_survive >= threshold).astype(str)#.replace({'True': 'True', 'False': 'False'})
    })


def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Transported
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
0027_01,True
0029_01,True
0032_01,True
0032_02,True
